In [1]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=4fe9c2dd07487ea33dfa784571c93fc3a6c6936841a7e2c67aa1a9eae7d2dc6f
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [2]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from collections import Counter #for Smote, 
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive

%cd /gdrive


trainfile = r'/gdrive/My Drive/CIS508/Assignment-3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508/Assignment-3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.dtypes)
train_data.head()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
(65000, 596)
CoverageField11A        int64
CoverageField11B        int64
CoverageField1A         int64
CoverageField1B         int64
CoverageField2A         int64
                        ...  
GeographicField64_CA    int64
GeographicField64_IL    int64
GeographicField64_NJ    int64
GeographicField64_TX    int64
QuoteConversion_Flag    int64
Length: 596, dtype: object


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
0,2,1,17,23,17,23,15,22,16,22,13,22,13,23,965.0,1.0200,23,0.9403,0.0006,-1,25,1,1,1,1,1,1,-1,7,1,2,1,2,13,22,-1,12,3,6,3,...,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0
1,5,9,6,8,6,8,5,7,5,8,13,22,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,8,...,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0
2,4,6,7,12,7,12,6,10,7,11,25,25,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,11,9,15,5,12,-1,12,10,16,6,14,2,13,-1,17,21,21,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0
3,15,23,3,2,3,2,2,2,3,2,13,22,13,23,1165.0,1.2665,10,0.9769,0.0004,-1,25,15,22,21,22,15,22,-1,15,24,23,9,18,2,3,-1,6,10,13,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0
4,4,6,8,13,8,13,7,11,7,13,13,22,13,23,1487.0,1.3045,23,0.9472,0.0006,-1,25,9,17,12,16,9,17,-1,22,9,8,10,20,2,6,-1,4,17,16,2,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0


In [4]:
print(test_data.shape)
print(test_data.dtypes)
test_data.head()

(173836, 596)
CoverageField11A         int64
CoverageField11B         int64
CoverageField1A          int64
CoverageField1B          int64
CoverageField2A          int64
                         ...  
GeographicField64_CA     int64
GeographicField64_IL     int64
GeographicField64_NJ     int64
GeographicField64_TX     int64
GeographicField64       object
Length: 596, dtype: object


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,GeographicField64
0,13,22,4,4,4,4,3,3,3,4,13,22,13,23,1487.0,1.3045,16,0.9364,0.0006,-1,25,9,18,13,18,9,18,-1,19,10,13,8,17,2,10,-1,20,19,18,25,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,IL
1,4,5,8,14,8,14,7,12,8,13,13,22,13,23,564.0,1.1886,11,0.9919,0.0038,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,17,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,NJ
2,3,3,11,18,11,18,10,16,10,18,13,22,13,23,564.0,1.0670,15,0.8945,0.0038,-1,25,4,10,10,16,5,10,-1,11,11,18,5,13,2,13,-1,15,21,20,3,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,NJ
3,5,9,14,22,15,22,13,20,22,25,13,22,13,23,1113.0,1.2665,21,0.8870,0.0004,-1,25,24,24,23,23,24,24,25,25,23,22,24,25,2,7,-1,13,11,14,9,...,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,TX
4,12,21,4,5,4,5,4,4,4,5,13,22,13,23,935.0,1.0200,25,0.9153,0.0007,-1,25,2,3,3,5,2,5,-1,7,6,6,2,6,15,23,-1,17,4,8,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,CA


In [5]:
Xtrain=train_data.iloc[:,:-1].copy()
Ytrain=train_data.iloc[:,-1].copy()

Xtest=test_data.iloc[:,:-1].copy()

print(Xtrain.shape)
print(Xtrain.dtypes)
Xtrain.head()

(65000, 595)
CoverageField11A        int64
CoverageField11B        int64
CoverageField1A         int64
CoverageField1B         int64
CoverageField2A         int64
                        ...  
GeographicField63_Y     int64
GeographicField64_CA    int64
GeographicField64_IL    int64
GeographicField64_NJ    int64
GeographicField64_TX    int64
Length: 595, dtype: object


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_Q,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX
0,2,1,17,23,17,23,15,22,16,22,13,22,13,23,965.0,1.0200,23,0.9403,0.0006,-1,25,1,1,1,1,1,1,-1,7,1,2,1,2,13,22,-1,12,3,6,3,...,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,1,0,0,0
1,5,9,6,8,6,8,5,7,5,8,13,22,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,8,...,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0
2,4,6,7,12,7,12,6,10,7,11,25,25,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,11,9,15,5,12,-1,12,10,16,6,14,2,13,-1,17,21,21,2,...,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0
3,15,23,3,2,3,2,2,2,3,2,13,22,13,23,1165.0,1.2665,10,0.9769,0.0004,-1,25,15,22,21,22,15,22,-1,15,24,23,9,18,2,3,-1,6,10,13,2,...,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1
4,4,6,8,13,8,13,7,11,7,13,13,22,13,23,1487.0,1.3045,23,0.9472,0.0006,-1,25,9,17,12,16,9,17,-1,22,9,8,10,20,2,6,-1,4,17,16,2,...,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0


In [6]:
Ytrain.head()

0    0
1    0
2    0
3    0
4    0
Name: QuoteConversion_Flag, dtype: int64

In [7]:
print(Xtest.shape)
print(Xtest.dtypes)
Xtest.head()

(173836, 595)
CoverageField11A        int64
CoverageField11B        int64
CoverageField1A         int64
CoverageField1B         int64
CoverageField2A         int64
                        ...  
GeographicField63_Y     int64
GeographicField64_CA    int64
GeographicField64_IL    int64
GeographicField64_NJ    int64
GeographicField64_TX    int64
Length: 595, dtype: object


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_Q,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX
0,13,22,4,4,4,4,3,3,3,4,13,22,13,23,1487.0,1.3045,16,0.9364,0.0006,-1,25,9,18,13,18,9,18,-1,19,10,13,8,17,2,10,-1,20,19,18,25,...,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0
1,4,5,8,14,8,14,7,12,8,13,13,22,13,23,564.0,1.1886,11,0.9919,0.0038,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,17,...,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0
2,3,3,11,18,11,18,10,16,10,18,13,22,13,23,564.0,1.0670,15,0.8945,0.0038,-1,25,4,10,10,16,5,10,-1,11,11,18,5,13,2,13,-1,15,21,20,3,...,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0
3,5,9,14,22,15,22,13,20,22,25,13,22,13,23,1113.0,1.2665,21,0.8870,0.0004,-1,25,24,24,23,23,24,24,25,25,23,22,24,25,2,7,-1,13,11,14,9,...,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0
4,12,21,4,5,4,5,4,4,4,5,13,22,13,23,935.0,1.0200,25,0.9153,0.0007,-1,25,2,3,3,5,2,5,-1,7,6,6,2,6,15,23,-1,17,4,8,2,...,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0


In [8]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(Ytrain))#before smote
sm = SMOTE(sampling_strategy='float', ratio=0.4)#no of minority /majority
Xres, Yres = sm.fit_resample(Xtrain, Ytrain)#after smote
print('Resampled dataset shape %s' % Counter(Yres))#after smote
#new training set with x n y numbers changed #we can do tjis before we do all the previous model building

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 21095})


In [9]:

#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(Xres, Yres)
clf_predict_train=clf.predict(Xres)


print("accuracy Score (training) for Decision TreE:{0:6f}".format(clf.score(Xres,Yres)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(Yres,clf_predict_train))
print("Classification report for Decision Tree")
print(classification_report(Yres,clf_predict_train))




accuracy Score (training) for Decision TreE:1.000000
Confusion Matrix for Decision Tree
[[52738     0]
 [    0 21095]]
Classification report for Decision Tree
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     52738
           1       1.00      1.00      1.00     21095

    accuracy                           1.00     73833
   macro avg       1.00      1.00      1.00     73833
weighted avg       1.00      1.00      1.00     73833



In [10]:

Ypred_test = pd.DataFrame(clf.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(clf.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results1_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results1_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [11]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=10)
clf_random.fit(Xres, Yres)
rand_parm=clf_random.best_params_
print(rand_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**rand_parm)
clf.fit(Xres,Yres)
clf_predict_train = clf.predict(Xres)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(Xres,Yres)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(Yres,clf_predict_train))
print("=== Classification Report after hypertuning for Decision Tree===")
print(classification_report(Yres,clf_predict_train))

#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(clf, Xres, Yres, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

{'min_samples_split': 10, 'min_samples_leaf': 7}
accuracy Score (training) after hypertuning for Decision Tree:0.962943
Confusion Matrix after hypertuning for Decision Tree
[[51694  1044]
 [ 1692 19403]]
=== Classification Report after hypertuning for Decision Tree===
              precision    recall  f1-score   support

           0       0.97      0.98      0.97     52738
           1       0.95      0.92      0.93     21095

    accuracy                           0.96     73833
   macro avg       0.96      0.95      0.95     73833
weighted avg       0.96      0.96      0.96     73833

=== All AUC Scores ===
[0.83969639 0.78467561 0.65555798 0.68273756 0.81536336 0.95024157
 0.93059114 0.50788055]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.7708430193164445


In [12]:
Ypred_test = pd.DataFrame(clf.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(clf.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results2_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results2_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [13]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(Xres, Yres)
rfc_predict_train=rfc.predict(Xres)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(Xres,Yres)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(Yres,rfc_predict_train))
print("classification report for Random Forest:")
print(classification_report(Yres,rfc_predict_train))

accuracy Score (training) for RandomForest:0.993567
Confusion Matrix for Random Forest:
[[52728    10]
 [  465 20630]]
classification report for Random Forest:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     52738
           1       1.00      0.98      0.99     21095

    accuracy                           0.99     73833
   macro avg       1.00      0.99      0.99     73833
weighted avg       0.99      0.99      0.99     73833



In [14]:
Ypred_test = pd.DataFrame(rfc.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(rfc.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results3_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results3_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [15]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,100,10),'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=10)
rfc_random.fit(Xres, Yres)
rand_parm_rfc=rfc_random.best_params_
print(rand_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**rand_parm_rfc)
rfc.fit(Xres,Yres)
rfc_predict_train = rfc.predict(Xres)
print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(Xres,Yres)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(Yres,rfc_predict_train))
print("=== Classification Report after hypertuning for Random Forest:===")
print(classification_report(Yres,rfc_predict_train))


#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, Xres, Yres, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


{'n_estimators': 50, 'min_samples_split': 80, 'min_samples_leaf': 5}
accuracy Score (training) after hypertuning for Random Forest:0.936126
Confusion Matrix after hypertuning for Random Forest:
[[52216   522]
 [ 4194 16901]]
=== Classification Report after hypertuning for Random Forest:===
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     52738
           1       0.97      0.80      0.88     21095

    accuracy                           0.94     73833
   macro avg       0.95      0.90      0.92     73833
weighted avg       0.94      0.94      0.93     73833

=== All AUC Scores ===
[0.93627053 0.93709109 0.93786012 0.93495454 0.94983267 0.99926285
 0.99921941 0.99382684]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9610397562765665


In [16]:
Ypred_test = pd.DataFrame(rfc.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(rfc.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results4_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results4_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [17]:
#Neural networks=============================================================
#Default mode
nn = MLPClassifier()
nn.fit(Xres, Yres)
nn_predict_train=nn.predict(Xres)
print("accuracy Score (training) for Neural network:{0:6f}".format(nn.score(Xres,Yres)))
print("Confusion Matrix for neural network:")
print(confusion_matrix(Yres,nn_predict_train))
print("=== Classification Report for neural network:===")
print(classification_report(Yres,nn_predict_train))

accuracy Score (training) for Neural network:0.856988
Confusion Matrix for neural network:
[[51334  1404]
 [ 9155 11940]]
=== Classification Report for neural network:===
              precision    recall  f1-score   support

           0       0.85      0.97      0.91     52738
           1       0.89      0.57      0.69     21095

    accuracy                           0.86     73833
   macro avg       0.87      0.77      0.80     73833
weighted avg       0.86      0.86      0.85     73833



In [18]:
Ypred_test = pd.DataFrame(nn.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(nn.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results5_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results5_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [42]:
 #Hyperparameter tuning for neural network
parameters={ 'activation': ['relu','logistic'], 'max_iter': range(100,150,10)}
nn_random = RandomizedSearchCV(nn,parameters,n_iter=10)
nn_random.fit(Xres, Yres)
rand_parm_nn=nn_random.best_params_
print(rand_parm_nn)

#contruct mlclassifier using the best parameters
nn = MLPClassifier(**rand_parm_nn)
nn.fit(Xres, Yres)
nn_predict_train = nn.predict(Xres)
print("accuracy Score (training) after hypertuning for Neural networks:{0:6f}".format(nn.score(Xres, Yres)))
print("Confusion Matrix after hypertuning for Neural network:")
print(confusion_matrix(Yres,nn_predict_train))
print("=== Classification Report after hypertuning for Neural network:===")
print(classification_report(Yres,nn_predict_train))

#run cross-validation on best parameters, get auc score
nn_cv_score = cross_val_score(nn, Xres, Yres, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(nn_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Neural network: ",nn_cv_score.mean())
 

{'max_iter': 100, 'activation': 'logistic'}
accuracy Score (training) after hypertuning for Neural networks:0.714288
Confusion Matrix after hypertuning for Neural network:
[[52738     0]
 [21095     0]]
=== Classification Report after hypertuning for Neural network:===
              precision    recall  f1-score   support

           0       0.71      1.00      0.83     52738
           1       0.00      0.00      0.00     21095

    accuracy                           0.71     73833
   macro avg       0.36      0.50      0.42     73833
weighted avg       0.51      0.71      0.60     73833

=== All AUC Scores ===
[0.32794047 0.32964974 0.33229488 0.46586916 0.46065613 0.45251311
 0.41285886 0.31686538]


=== Mean AUC Score ===
Mean AUC Score - Neural network:  0.38733096597120276


In [43]:
Ypred_test = pd.DataFrame(nn.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(nn.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results6_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results6_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [21]:
#SVC=============================================================
#Default mode
svc = LinearSVC()
svc.fit(Xres, Yres)
svc_predict_train=svc.predict(Xres)
print("accuracy Score (training) for svc:{0:6f}".format(svc.score(Xres, Yres)))
print("Confusion Matrix for svc:")
print(confusion_matrix(Yres,svc_predict_train))
print("=== Classification Report for svc:===")
print(classification_report(Yres,svc_predict_train))

accuracy Score (training) for svc:0.361952
Confusion Matrix for svc:
[[ 6193 46545]
 [  564 20531]]
=== Classification Report for svc:===
              precision    recall  f1-score   support

           0       0.92      0.12      0.21     52738
           1       0.31      0.97      0.47     21095

    accuracy                           0.36     73833
   macro avg       0.61      0.55      0.34     73833
weighted avg       0.74      0.36      0.28     73833



In [22]:
Ypred_test = pd.DataFrame(svc.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#Ypred_prob=pd.DataFrame(svc.predict_proba(Xtest))
#Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results7_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results7_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,1
1,5,1
2,7,1
3,9,0
4,10,1


In [23]:
#Hyperparameter tuning for svc
parameters={ 'max_iter': range(10,80,10),'loss':['hinge','squared_hinge']} #'decision_function_shape': ['ovo','ovr']
svc = LinearSVC()
svc_random = RandomizedSearchCV(svc,parameters,n_iter=10)
svc_random.fit(Xres, Yres)
rand_parm_svc=svc_random.best_params_
print(rand_parm_svc)

#contruct svc  using the best parameters
svc = LinearSVC(**rand_parm_svc)
svc.fit(Xres, Yres)
svc_predict_train = svc.predict(Xres)
print("accuracy Score (training) after hypertuning for svc:{0:6f}".format(svc.score(Xres, Yres)))
print("Confusion Matrix after hypertuning for svc:")
print(confusion_matrix(Yres,svc_predict_train))
print("=== Classification Report after hypertuning for svc:===")
print(classification_report(Yres,svc_predict_train))

#run cross-validation on best parameters, get auc score
svc_cv_score = cross_val_score(svc, Xres, Yres, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(svc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - svc: ",svc_cv_score.mean())

{'max_iter': 70, 'loss': 'hinge'}
accuracy Score (training) after hypertuning for svc:0.714992
Confusion Matrix after hypertuning for svc:
[[52525   213]
 [20830   265]]
=== Classification Report after hypertuning for svc:===
              precision    recall  f1-score   support

           0       0.72      1.00      0.83     52738
           1       0.55      0.01      0.02     21095

    accuracy                           0.71     73833
   macro avg       0.64      0.50      0.43     73833
weighted avg       0.67      0.71      0.60     73833

=== All AUC Scores ===
[0.57791617 0.68549236 0.62951904 0.96837737 0.74122989 0.90788624
 0.46797984 0.76033102 0.89780237 0.8917874 ]


=== Mean AUC Score ===
Mean AUC Score - svc:  0.7528321700190561


In [24]:
Ypred_test = pd.DataFrame(svc.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#Ypred_prob=pd.DataFrame(svc.predict_proba(Xtest))
#Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results8_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results8_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [25]:
#KNN=============================================================
#Default mode
knn = KNeighborsClassifier()
knn.fit( Xres, Yres)
knn_predict_train=knn.predict(Xres)
print("accuracy Score (training) for knn:{0:6f}".format(knn.score( Xres, Yres)))
print("Confusion Matrix for knn:")
print(confusion_matrix(Yres,knn_predict_train))
print("=== Classification Report for knn:===")
print(classification_report(Yres,knn_predict_train))

accuracy Score (training) for knn:0.823290
Confusion Matrix for knn:
[[47243  5495]
 [ 7552 13543]]
=== Classification Report for knn:===
              precision    recall  f1-score   support

           0       0.86      0.90      0.88     52738
           1       0.71      0.64      0.67     21095

    accuracy                           0.82     73833
   macro avg       0.79      0.77      0.78     73833
weighted avg       0.82      0.82      0.82     73833



In [26]:
Ypred_test = pd.DataFrame(knn.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(knn.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results9_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results9_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [29]:
#Hyperparameter tuning for knn
parameters={ 'n_neighbors': [3,5,7], 'leaf_size': [20,30,40]}
knn_random = RandomizedSearchCV(knn,parameters,n_iter=10)
knn_random.fit(Xres, Yres)
rand_parm_knn=knn_random.best_params_
print(rand_parm_knn)

#contruct knn  using the best parameters
knn = KNeighborsClassifier(**rand_parm_knn)
knn.fit(Xres, Yres)
knn_predict_train = knn.predict(Xres)
print("accuracy Score (training) after hypertuning for knn:{0:6f}".format(knn.score(Xres, Yres)))
print("Confusion Matrix after hypertuning for knn:")
print(confusion_matrix(Yres,knn_predict_train))
print("=== Classification Report after hypertuning for knn:===")
print(classification_report(Yres,knn_predict_train))

#run cross-validation on best parameters, get auc score
knn_cv_score = cross_val_score(knn, Xres, Yres, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(knn_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - svc: ",knn_cv_score.mean())

{'n_neighbors': 3, 'leaf_size': 20}
accuracy Score (training) after hypertuning for knn:0.869963
Confusion Matrix after hypertuning for knn:
[[47626  5112]
 [ 4489 16606]]
=== Classification Report after hypertuning for knn:===
              precision    recall  f1-score   support

           0       0.91      0.90      0.91     52738
           1       0.76      0.79      0.78     21095

    accuracy                           0.87     73833
   macro avg       0.84      0.85      0.84     73833
weighted avg       0.87      0.87      0.87     73833

=== All AUC Scores ===
[0.33068217 0.14131566 0.06735077 0.06223894 0.10793987 0.20356419
 0.19410964 0.18331795]


=== Mean AUC Score ===
Mean AUC Score - svc:  0.16131489860520937


In [30]:
Ypred_test = pd.DataFrame(knn.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(knn.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results10_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results10_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [32]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using Neural networks,support vector machine, knn, RandomForest and Decision Tree Classifier\n")

models = [LinearSVC() , RandomForestClassifier(), DecisionTreeClassifier() ]#we r giving models as arguments we can also give those parameters like grid_params
      #we are giving  data after smote and we did smote only for train test
Strain, Stest = stacking(models,                   
                           Xres, Yres, Xtest,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using Neural networks,support vector machine, knn, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LinearSVC]
    fold  0:  [0.59873233]
    fold  1:  [0.54612926]
    fold  2:  [0.71659985]
    fold  3:  [0.64647559]
    ----
    MEAN:     [0.62698426] + [0.06274267]
    FULL:     [0.62698251]

model  1:     [RandomForestClassifier]
    fold  0:  [0.90682052]
    fold  1:  [0.90113224]
    fold  2:  [0.90719471]
    fold  3:  [0.90052555]
    ----
    MEAN:     [0.90391825] + [0.00309962]
    FULL:     [0.90391830]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.89506474]
    fold  1:  [0.89414378]
    fold  2:  [0.89917651]
    fold  3:  [0.89456575]
    ----
    MEAN:     [0.89573769] + [0.00201199]
    FULL:     [0.895

In [33]:
#STACKING - CONTRUCT A decision tree classifier MODEL==============================
model = DecisionTreeClassifier()
    
model = model.fit(Strain, Yres)
Ypred_train = model.predict(Strain)
print('Final prediction score for ensemble methods using decision tree classifier: [%.8f]' % accuracy_score(Yres, Ypred_train))
print("Confusion Matrix after STACKING for decision tree classifier :")
print(confusion_matrix(Yres, Ypred_train))
print("=== Classification Report after STACKING for decision tree classifier===")
print(classification_report(Yres, Ypred_train))

Ypred_test = pd.DataFrame(model.predict(Stest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#we are doing one layer with decision tree classifier

Final prediction score for ensemble methods using decision tree classifier: [0.90451424]
Confusion Matrix after STACKING for decision tree classifier :
[[51742   996]
 [ 6054 15041]]
=== Classification Report after STACKING for decision tree classifier===
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     52738
           1       0.94      0.71      0.81     21095

    accuracy                           0.90     73833
   macro avg       0.92      0.85      0.87     73833
weighted avg       0.91      0.90      0.90     73833



,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [34]:
Ypred_prob=pd.DataFrame(model.predict_proba(Stest))
Ypred_prob.head()

,0,1
0,0.956144,0.043856
1,0.956144,0.043856
2,0.956144,0.043856
3,0.956144,0.043856
4,0.956144,0.043856


In [35]:
pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results_DT_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results_DT_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [36]:
#STACKING - CONTRUCT A GradientBoostingClassifier MODEL==============================
model = GradientBoostingClassifier()
    
model = model.fit(Strain, Yres)
Ypred_train = model.predict(Strain)
print('Final prediction score for ensemble methods using GradientBoostingClassifier: [%.8f]' % accuracy_score(Yres, Ypred_train))
print("Confusion Matrix after STACKING for GradientBoostingClassifier() :")
print(confusion_matrix(Yres, Ypred_train))
print("=== Classification Report after STACKING for GradientBoostingClassifier()r===")
print(classification_report(Yres, Ypred_train))

Ypred_test = pd.DataFrame(model.predict(Stest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#we are doing one layer with GradientBoostingClassifier()

Final prediction score for ensemble methods using GradientBoostingClassifier: [0.90451424]
Confusion Matrix after STACKING for GradientBoostingClassifier() :
[[51742   996]
 [ 6054 15041]]
=== Classification Report after STACKING for GradientBoostingClassifier()r===
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     52738
           1       0.94      0.71      0.81     21095

    accuracy                           0.90     73833
   macro avg       0.92      0.85      0.87     73833
weighted avg       0.91      0.90      0.90     73833



,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [37]:
Ypred_prob=pd.DataFrame(model.predict_proba(Stest))
Ypred_prob.head()

,0,1
0,0.956137,0.043863
1,0.956137,0.043863
2,0.956137,0.043863
3,0.956137,0.043863
4,0.956137,0.043863


In [38]:
pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results_GB_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results_GB_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [39]:
#STACKING - CONTRUCT A random forest classfier MODEL==============================
model = RandomForestClassifier()
    
model = model.fit(Strain, Yres)
Ypred_train = model.predict(Strain)
print('Final prediction score for ensemble methods using random forest: [%.8f]' % accuracy_score(Yres, Ypred_train))
print("Confusion Matrix after STACKING for decision tree classifier :")
print(confusion_matrix(Yres, Ypred_train))
print("=== Classification Report after STACKING for random forestr===")
print(classification_report(Yres, Ypred_train))

Ypred_test = pd.DataFrame(model.predict(Stest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#we are doing one layer with random forest classifier

Final prediction score for ensemble methods using random forest: [0.90451424]
Confusion Matrix after STACKING for decision tree classifier :
[[51742   996]
 [ 6054 15041]]
=== Classification Report after STACKING for random forestr===
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     52738
           1       0.94      0.71      0.81     21095

    accuracy                           0.90     73833
   macro avg       0.92      0.85      0.87     73833
weighted avg       0.91      0.90      0.90     73833



,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [40]:
Ypred_prob=pd.DataFrame(model.predict_proba(Stest))
Ypred_prob.head()

,0,1
0,0.956103,0.043897
1,0.956103,0.043897
2,0.956103,0.043897
3,0.956103,0.043897
4,0.956103,0.043897


In [41]:
pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results_RF_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/SMOTE4/Results_RF_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0
